# Init

In [ ]:
import requests
import json
import common
import os
CONF = common.GrConf()
txCaller = common.GrGetAPI(CONF,'stock_log')

# Workflow to import Grocy transaction data
User input to get transaction window and then call

In [ ]:
startDate = input('Enter start date YYYY-MM-DD')
timeQuery = input("Enter start time (UTC) or blank for 00:00:00")
if timeQuery:
    startTime = timeQuery

else:
    startTime = '00:00:00'

#To do, end time
timeParam = startDate+" "+startTime
print(timeParam)

In [ ]:
txCaller.buildParam("row_created_timestamp>="+timeParam)

txCaller.call()
r=txCaller.r
if not "200" in str(r):
    print(f'Call failed :( Error:{str(r)}')
else:
    print('Success')

# Save offline copy (limited to success) to implement

In [ ]:
if "200" in str(r):
    resp = input('Confirm save offline copy?')
    if "y" in resp or "Y" in resp:        
        if os.path.exists("localgrocyTxdata.json"):
            record=open("localgrocyTxdata.json",mode="w")
        else:
            record=open("localgrocyTxdata.json",mode="x")

        json.dump(r.json(),record)
        record.close()
        print("localgrocyTxdata.json was saved")

# Mapping
Loop through the relevant interfaces that provide names to IDs
Todo: adapt this to also save the mapping locally for building mass uploads if needed

In [ ]:
callThese = ['products','locations','shopping_locations','users']
columnMap = {'products':['product_id'],
             'locations':['location_id'],
             'shopping_locations':['shopping_location_id'],
             'users':['user_id']}
paramMap = {'products':"active=1",
             'locations':"active=1",
             'shopping_locations':"active=1",
             'users':""}
sourceMap = {'products':"name",
             'locations':"name",
             'shopping_locations':"name",
             'users':"username"}

rjson = r.json()
for a in callThese:

    iterCaller = common.GrGetAPI(CONF,a)
    if paramMap[a]:
        iterCaller.buildParam(paramMap[a])
    iterCaller.call()
    if "200" not in str(iterCaller.r):
        print(a+' API failure')
        print(iterCaller.r.url)


# Join the API calls so far into one rjson master file

    for x in rjson:
        for y in iterCaller.r.json():
                for z in range(len(columnMap[a])):
                    #print(z)
                    #print(columnMap[a])
                    #print(columnMap[a][z])
                    #print(x['location_id'])
                    if x[columnMap[a][z]] == y['id']:
                        x[columnMap[a][z]] = y[sourceMap[a]]
                        


# Tabulate Data
Input response to DataFrame & Display as an ITable

In [ ]:
import pandas
import io
import dateutil
from itables import init_notebook_mode
from itables import show


init_notebook_mode(all_interactive=True)
txFrame = pandas.DataFrame(rjson)
# Todo: timestamps are important here. Convert to np times and convert UTC to local

txFrame['row_created_timestamp'] = pandas.to_datetime(txFrame['row_created_timestamp'],utc=True)
txFrame['row_created_timestamp'] = txFrame['row_created_timestamp'].dt.tz_convert(CONF.tz)

show(txFrame,buttons=["copyHtml5", "csvHtml5", "excelHtml5"])